In [1]:
using DrWatson
@quickactivate "dare"

using ReinforcementLearning
using IntervalSets
using LinearAlgebra
using ControlSystems
using CUDA
using DataFrames
using PlotlyJS
# using Plots
# using Interpolations



include(srcdir("nodeconstructor.jl"))
include(srcdir("LookupTables.jl"))
include(srcdir("env.jl"));

using .LookupTables

WebIO._IJuliaInit()

## Define Battery moduls

In [2]:
Base.@kwdef mutable struct battery_module
     R = [0.010342 0.0012244;
        0.0067316 0.0011396;
        0.0051156 0.0012661;
        0.0043447 0.0012265;
        0.0038826 0.0011163;
        0.0034226 0.0009968;
        0.003346 0.0011458;
        0.0033222 0.001345;
        0.0033201 0.0013091;
        0.0032886 0.0010986;
        0.0028114 0.0010309]     # Resistance for n-th RC pair
    C = [2287.7 11897;
        6122 24515;
        18460 42098;
        20975 44453;
        15254 33098;
        10440 24492;
        13903 32975;
        16694 40007;
        15784 35937;
        12165 26430;
        9118 24795]   # Capacitance for n-th RC pair
end;

In [3]:
Base.@kwdef mutable struct battery_block
    battery_module::battery_module
    T = 273.15    # temperature offset
    n = 3                      # number of RC pairs
    R_0 = 0.016                  # series resistance
    V_0 = [3.5042 3.5136;
        3.5573 3.5646;
        3.6009 3.6153;
        3.6393 3.6565;
        3.6742 3.6889;
        3.7121 3.7214;
        3.7937 3.8078;
        3.8753 3.8945;
        3.97 3.9859;
        4.0764 4.0821;
        4.1924 4.193]                  # Battery open-circuit voltage
    Q = 0                      # Current Ladung
    Q_0 = 26*3600              # Nominal Ladung
    SOC = 0                    # State of Charge (SOC)
    SOC_BP = vec([0 0.1 0.2 0.3 0.4 0.5 0.6 0.7 0.8 0.9 1])
    T_BP = vec([293.15 313.15])
    tau = 1 #0.1e-3
    mode = nothing
    V = 0
    LT_V0 = LookupTable2d(SOC_BP,T_BP,V_0)
    LT_R = LookupTable2d(SOC_BP,T_BP,battery_module.R)
    
end;

In [4]:
function update(self::battery_block, I_batt, T)
    
    T_ = self.T + T
    
    if sign(I_batt) == -1
        self.mode = "charge"
    else
        self.mode = "discharge"
    end
    
    self.Q = clamp(self.Q -I_batt*self.tau,0,self.Q_0) # Clamp 
        
    self.SOC = clamp(self.Q/self.Q_0,0,1)
    
    SOC_round = round(self.SOC;digits = 1)
    
    idx_soc = findall(x->x==SOC_round, self.SOC_BP)[1]
    
    idx_t = 1
    
    # self.V = self.V_0[idx_soc, idx_t] - self.n * self.battery_module.R[idx_soc,idx_t] * I_batt - self.R_0 * I_batt
    
    self.V = lookup(self.SOC,T_,self.LT_V0) - self.n * lookup(self.SOC,T_,self.LT_R) * I_batt - self.R_0 * I_batt
    
    
end

update (generic function with 1 method)

In [5]:
Cell = battery_module();

In [6]:
Battery = battery_block(battery_module=Cell);

### Test

In [7]:
SOC = []
Voltage = []
Current = []

Any[]

In [8]:
update(Battery, 200.0, 20)

-5.901000000000001

In [9]:
for i in 1:20
    I_in = -1000

    update(Battery, I_in, 20)

    append!(SOC, Battery.SOC)
    append!(Current, I_in)
    append!(Voltage, Battery.V)
    
    println("Mode: $(Battery.mode)")
    println("State of Charge $(Battery.SOC)")
    println("Output Voltage $(Battery.V)")
end

Mode: charge
State of Charge 0.010683760683760684
Output Voltage 49.37869358974359
Mode: charge
State of Charge 0.021367521367521368
Output Voltage 48.22718717948718
Mode: charge
State of Charge 0.03205128205128205
Output Voltage 47.07568076923077
Mode: charge
State of Charge 0.042735042735042736
Output Voltage 45.924174358974355
Mode: charge
State of Charge 0.053418803418803416
Output Voltage 44.77266794871795
Mode: charge
State of Charge 0.0641025641025641
Output Voltage 43.621161538461536
Mode: charge
State of Charge 0.07478632478632478
Output Voltage 42.46965512820513
Mode: charge
State of Charge 0.08547008547008547
Output Voltage 41.318148717948716
Mode: charge
State of Charge 0.09615384615384616
Output Voltage 40.16664230769231
Mode: charge
State of Charge 0.10683760683760683
Output Voltage 39.42359401709402
Mode: charge
State of Charge 0.11752136752136752
Output Voltage 38.91030341880342
Mode: charge
State of Charge 0.1282051282051282
Output Voltage 38.39701282051282
Mode: charg

In [22]:
x1 = [1,2,3,4,5]
y1 = [2,2,4,4,2];
x2 = [1,2,3,4,5,6,7,8]
y2 = [2,2,4,4,5,5,5,1];

In [26]:
plot([scatter(y=SOC), scatter(y=Voltage)])

data: [
  "scatter with fields type and y",
  "scatter with fields type and y"
]

layout: "layout with fields margin and template"